In [4]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [6]:
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

In [7]:
import tensorflow as tf

n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [8]:

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [11]:
import numpy as np
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name): #give Tensorboad a better look
        n_inputs = int(X.get_shape()[1]) #input matrix shape 
        stddev = 2 / np.sqrt(n_inputs) # make weight a truncated guassian distribution, it will make matrix converge faster
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel") 
        b = tf.Variable(tf.zeros([n_neurons]), name="bias") #bias
        Z = tf.matmul(X, W) + b #subgraph 
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [12]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [13]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [14]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [15]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [16]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [18]:
#Execution stage
n_epochs = 40
batch_size = 50
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                            y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final.ckpt")

(0, 'Train accuracy:', 0.83999997, 'Test accuracy:', 0.90780002)
(1, 'Train accuracy:', 0.89999998, 'Test accuracy:', 0.92979997)
(2, 'Train accuracy:', 0.98000002, 'Test accuracy:', 0.9364)
(3, 'Train accuracy:', 1.0, 'Test accuracy:', 0.94279999)
(4, 'Train accuracy:', 0.89999998, 'Test accuracy:', 0.94840002)
(5, 'Train accuracy:', 0.95999998, 'Test accuracy:', 0.95249999)
(6, 'Train accuracy:', 0.95999998, 'Test accuracy:', 0.95469999)
(7, 'Train accuracy:', 0.94, 'Test accuracy:', 0.95810002)
(8, 'Train accuracy:', 0.92000002, 'Test accuracy:', 0.9605)
(9, 'Train accuracy:', 0.98000002, 'Test accuracy:', 0.9623)
(10, 'Train accuracy:', 1.0, 'Test accuracy:', 0.96350002)
(11, 'Train accuracy:', 0.98000002, 'Test accuracy:', 0.96530002)
(12, 'Train accuracy:', 0.98000002, 'Test accuracy:', 0.9673)
(13, 'Train accuracy:', 0.98000002, 'Test accuracy:', 0.96810001)
(14, 'Train accuracy:', 0.95999998, 'Test accuracy:', 0.96890002)
(15, 'Train accuracy:', 0.98000002, 'Test accuracy:', 0.